# Run the `snowBMI` model using Basic Model Interface commands

This notebook demonstrates the execution of a simple temperature index snow model using BMI. View the source code for the [model](https://github.com/SnowHydrology/snowBMI/blob/master/snow/snow.py) and its [BMI](https://github.com/SnowHydrology/snowBMI/blob/master/snow/bmi_snow.py) on GitHub.

This exercise is based on the [heat](https://github.com/csdms/bmi-example-python) example from CSDMS.

Start by importing `os`, `numpy` and the `Snow` BMI:

In [1]:
import os
import numpy as np

from snow import SnowBmi

Create an instance of the model's BMI.

In [2]:
x = SnowBmi()

What's the name of this model?

In [3]:
print(x.get_component_name())

Temperature Index Snow Model with BMI


Start the `Snow` model through its BMI using a configuration file:

In [4]:
cat snow.yaml

# Snow model configuration
rs_method: 1         # 1 = single threshold, 2 = dual threshold
rs_thresh: 2.5       # rain-snow temperature threshold when rs_method = 1 (°C)
snow_thresh_max: 1.5 # maximum all-snow temp when rs_method = 2 (°C)
rain_thresh_min: 4.5 # minimum all-rain temp when rs_method = 2 (°C)
ddf_max: 1           # maximum degree day melt factor (mm/day/°C)
ddf_min: 0           # minimum degree day melt factor (mm/day/°C)
tair_melt_thresh: 1  # air temperature threshold above which melt can occur (°C)
swe_init: 0          # initial snow water equivalent (mm)
dayofyear: 274       # Day of year of simulation start (ex: 1 = Jan 1, 274 = Oct 1)

In [5]:
x.initialize("snow.yaml")

Check the time information for the model.

In [6]:
print("Start time:", x.get_start_time())
print("End time:", x.get_end_time())
print("Current time:", x.get_current_time())
print("Time step:", x.get_time_step())
print("Time units:", x.get_time_units())

Start time: 0.0
End time: 1.7976931348623157e+308
Current time: 0.0
Time step: 86400
Time units: s


Show the input and output variables for the component (aside on [Standard Names](https://csdms.colorado.edu/wiki/CSDMS_Standard_Names)):

In [7]:
print(x.get_input_var_names())
print(x.get_output_var_names())

('atmosphere_water__precipitation_leq-volume_flux', 'land_surface_air__temperature')
('snowpack__liquid-equivalent_depth', 'snowpack__melt_volume_flux')


For a quick example, set air temperature and precipitation to a constant value.

In [8]:
air_temperature = np.zeros(1,)
print(air_temperature)
air_temperature[0,] = -5. #°C
print(air_temperature)
temp_array = np.zeros(1,)
x.get_value('land_surface_air__temperature', temp_array)
print(temp_array.shape)
print(x.get_value_ptr('snowpack__liquid-equivalent_depth'))
val = x.get_value_ptr('land_surface_air__temperature')
print(val)
print(x._values['land_surface_air__temperature'])
precipitation = 10 #mm

[0.]
[-5.]
(1,)
[0.]
[0.]
[0.]


In [9]:
x.set_value("land_surface_air__temperature", air_temperature)
print("air temp = ", x.get_value_ptr('land_surface_air__temperature'))
print(x._values)

air temp =  [-5.]
{'atmosphere_water__precipitation_leq-volume_flux': array([0.]), 'land_surface_air__temperature': array([-5.]), 'snowpack__liquid-equivalent_depth': array([0.]), 'snowpack__melt_volume_flux': array([0.])}


Next, get the identifier for the grid on which the temperature variable is defined:

In [ ]:
grid_id = x.get_var_grid("plate_surface__temperature")
print("Grid id:", grid_id)

Then get the grid attributes:

In [ ]:
print("Grid type:", x.get_grid_type(grid_id))

rank = x.get_grid_rank(grid_id)
print("Grid rank:", rank)

shape = np.ndarray(rank, dtype=int)
x.get_grid_shape(grid_id, shape)
print("Grid shape:", shape)

spacing = np.ndarray(rank, dtype=float)
x.get_grid_spacing(grid_id, spacing)
print("Grid spacing:", spacing)

These commands are made somewhat un-Pythonic by the generic design of the BMI.

Through the model's BMI, zero out the initial temperature field, except for an impulse near the middle.
Note that *set_value* expects a one-dimensional array for input.

In [ ]:
temperature = np.zeros(shape)
temperature[3, 4] = 100.0
x.set_value("plate_surface__temperature", temperature)

Check that the temperature field has been updated. Note that *get_value* expects a one-dimensional array to receive output.

In [12]:
temperature_flat = np.empty_like(temperature).flatten()
x.get_value("plate_surface__temperature", temperature_flat)
print(temperature_flat.reshape(shape))

AttributeError: 'int' object has no attribute 'flatten'

Now advance the model by a single time step:

In [ ]:
x.update()

View the new state of the temperature field:

In [ ]:
x.get_value("plate_surface__temperature", temperature_flat)
print(temperature_flat.reshape(shape))

There's diffusion!

Advance the model to some distant time:

In [ ]:
distant_time = 2.0
while x.get_current_time() < distant_time:
    x.update()

View the final state of the temperature field:

In [ ]:
np.set_printoptions(formatter={"float": "{: 5.1f}".format})
x.get_value("plate_surface__temperature", temperature_flat)
print(temperature_flat.reshape(shape))

Note that temperature isn't conserved on the plate:

In [ ]:
print(temperature_flat.sum())

End the model:

In [ ]:
x.finalize()